# Nixtla Tutorial

The Flash team is excited to share with you a small tutorial on Nixtla.
Before jumping into this tutorial, we recommend giving a look to this [README](README.md) in order to get more familiar with Nixtla and its pros/cons ! 

Now that’s being said, let’s dig into a small example where we will use the M4 competition hourly dataset to forecast the next 24 next hours.

To do so we'll explore the following features in Nixtla:

1. Define Statistical models using the `StatsForecast` Nixtla package,
2. Evaluate the model's performance using Cross Validation,
3. Implement ML models (ex. LGBM) using the `MLForecast` Nixtla package
4. [Bonus] Generate confidence intervals using Conformal predictions

## Import libraries

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datasetsforecast.m4 import M4
from statsforecast import StatsForecast
import random
import os
from utils import evaluate_cross_validation, get_best_model_forecast, hour_index

random.seed(0)
os.environ['NIXTLA_ID_AS_COL'] = '1'

## Import data

In [ ]:
await M4.async_download('data', group='Hourly')
train_df, *_ = M4.load('data', 'Hourly')


For the sake of running a faster training, we will randomly pick 4 UIDs only:

In [ ]:
uids = train_df['unique_id'].unique()
sample_uids = random.choices(uids, k=4)

In [ ]:
train_df = train_df[train_df['unique_id'].isin(sample_uids)].reset_index(drop=True)
train_df = train_df.groupby('unique_id').tail(7 * 24) 
train_df['ds'] = train_df['ds'].astype('int64')

In [ ]:
StatsForecast.plot(train_df, max_insample_length=24 * 14, engine="plotly")

## Baseline definition with StatsForecast

As a baseline, we define the following models:

- Historical Average: Arithmetic mean
- The AutoARIMA model: An implementation of the ARIMA model that uses an automatic process to select the optimal ARIMA (Autoregressive Integrated Moving Average) model parameters for a given time series.

Before using those models, we need to adhere to some naming conventions:
- [unique_id] for the time series identifier, 
- [ds] for the date, 
- [y] for the target variable.


In [ ]:
from statsforecast.models import (
    HistoricAverage,
    AutoARIMA
    )

In [ ]:
HORIZON = 24
EXPECTED_SEASONAL_LENGTH = 24

In [ ]:
models = [
    HistoricAverage(),
    AutoARIMA(season_length=EXPECTED_SEASONAL_LENGTH)
]

wrapper_models = StatsForecast( 
    models=models,
    freq=1, # Refers to the datestamps in your data. 1 or 'H' for hourly data, 'M' for monthly, etc ... 
    n_jobs=-1,
)

Let's try to forecast the next 24 hours:

In [ ]:
forecasts_df = wrapper_models.forecast(df=train_df, h=HORIZON)

In [ ]:
StatsForecast.plot(train_df, forecasts_df, models=["HistoricAverage","AutoARIMA"], engine = 'plotly')

We can also generate confidence intervals using the `level` parameter:

In [ ]:
CONFIDENCE_LEVEL = 90
probabilistic_forecasts_df = wrapper_models.forecast(df=train_df, h=HORIZON, level=[CONFIDENCE_LEVEL])
StatsForecast.plot(train_df, probabilistic_forecasts_df, models=["AutoARIMA"], level=[CONFIDENCE_LEVEL], engine = 'plotly')


## Model evaluation
We can use the cross-validation function to backtest our models.

In [ ]:
crossvalidation_df = wrapper_models.cross_validation(
    df = train_df,
    h = HORIZON,
    step_size = HORIZON, # Step size between each window
    n_windows = 4,       # Number of windows used for cross validation.
    level = [CONFIDENCE_LEVEL]
)

In [ ]:
crossvalidation_df

Let's evaluate our model now!

In [ ]:
evaluation_df = evaluate_cross_validation(crossvalidation_df, level = [CONFIDENCE_LEVEL])
evaluation_df


In [ ]:
evaluation_df[["metric", "AutoARIMA", "HistoricAverage"]].groupby('metric', as_index=False).mean()

In [ ]:
summary_df = evaluation_df.groupby(['metric','best_model']).size().sort_values().to_frame()
summary_df

In [ ]:
seasonal_ids = (
    evaluation_df
    .query('metric == "coverage_level90"')
    .query('best_model == "HistoricAverage"')
    .unique_id
    .drop_duplicates()
)

StatsForecast.plot(train_df,crossvalidation_df, unique_ids=seasonal_ids, models=["AutoARIMA","HistoricAverage"], level=[CONFIDENCE_LEVEL], plot_random=False)

## Beat the baseline !

Let's try to beat our baseline. We'll use available  MLForecast tools to compute:
- lags, 
- moving averages (mean, min, max, standard deviation, exponential mean, etc.), 
- seasonal and moving averages, 
- seasonal variables, 
- and manipulate the target variable (BoxCox, differentiation, scaling, etc.).

We'll also use a LightGBM model and a very naive custom model that predicts previous value.

In [ ]:
from mlforecast import MLForecast
from mlforecast.lag_transforms import ExpandingMean, RollingMean
from mlforecast.target_transforms import Differences
import lightgbm as lgb 
from sklearn.base import BaseEstimator

We will start by pre-processing our data to:
- Transform our Target values by subtracting the value from the same hour in the previous day (The goal is to remove the strong seasonality on the hour of the day)
- Create lag features and rolling averages
- Extract date features such as the hour of the day

Pre-processing steps can be applied on the data withou training phase using `preprocess` function:

In [ ]:
forecaster = MLForecast(
    models=[],
    freq=1,
    target_transforms=[Differences([24])],
    lags=[1, 24],
    lag_transforms={
        1: [ExpandingMean()],
        24: [RollingMean(window_size=48)],
    },
    date_features=[hour_index],
)

prep_train_df = forecaster.preprocess(train_df)
prep_train_df

We will now train our model using a Naive Forecasted we define and an LGBM:

In [ ]:
class Naive(BaseEstimator):
    def fit(self,X, y):
        return self

    def predict(self, X):
        return X['lag1']
    

In [ ]:
model_params = {
    'verbosity': -1,
    'num_leaves': 512,
}

ml_forecaster = MLForecast(
    models=[
        Naive(),
        lgb.LGBMRegressor(**model_params),
    ],
    freq=1,
    target_transforms=[Differences([24])],
    lags=[1, 24],
    lag_transforms={
        1: [ExpandingMean()],
        24: [RollingMean(window_size=48)],
    },
    date_features=[hour_index],
)

#ml_forecaster.fit(train_df)

In [ ]:
ml_forecast = ml_forecaster.predict(h=HORIZON)

In [ ]:
StatsForecast.plot(train_df,ml_forecast)

# Bonus: Conformal prediction

Multi-quantile losses and statistical models can provide prediction intervals, but the problem is that these are uncalibrated, meaning that the actual frequency of observations falling within the interval does not align with the confidence level associated with it. Statistical methods also assume normality. 

Conformal prediction intervals use cross-validation on a point forecaster model to generate the intervals. This means that no prior probabilities are needed, and the output is well-calibrated. 

No additional training is needed, and the model is treated as a black box. The approach is compatible with any model.

In [ ]:
from mlforecast.utils import PredictionIntervals

In [ ]:
ml_forecaster.fit(
    train_df,
    static_features=[],
    prediction_intervals=PredictionIntervals(n_windows=3, h=HORIZON, method="conformal_distribution")
)

In [ ]:
ml_forecast = ml_forecaster.predict(h=HORIZON, level=[CONFIDENCE_LEVEL])
StatsForecast.plot(train_df, ml_forecast, level=[CONFIDENCE_LEVEL])